<div style="text-align: center;">
    <span style="font-weight: bold;">Universidad de Buenos Aires</span>
<br/>
    Materia: <span style="font-weight: bold;">Tecnologías emergentes</span><br/>
<br/>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f9/UBA.svg/1200px-UBA.svg.png" width="150"/>
<br/>
<br/>
    Trabajo práctico sobre <span style="font-weight: bold;">Machine Learning</span><br/>
    Alumno: <span style="font-weight: bold;">Mariano La Penna</span><br/>
Padrón 98.432<br/>
<br/>
Diciembre de 2022<br/>
</div>

<span style="font-weight: bold;">Descripción e intención</span><br/><br/>
El presente trabajo práctico, de carácter individual, consiste en haber replicado un notebook público que resuelve una competencia de Kaggle de detección de tejidos linfáticos tumorales, e intercalarle de principio a fin explicaciones en español de las sentencias de código. Dicha tarea implicó el aprendizaje de un montón de asuntos: librerías, métodos, atributos, sintaxis, etc. y explicarlos en un lenguaje entre "técnico" y "criollo" :)<br/><br/>
El propósito no es solo cumplir con la práctica de un contenido de la materia, sino, de poner luego en dominio público un notebook de un asunto bastante complejo, listo para ser leído por principiantes, aspirando a que puedan lograr su total comprensión, ya que más de una vez me he encontrado tratando de entender código ajeno, o incluso replicándolo yo, sin saber la mayoría de las líneas qué función cumplían, y se demora bastante tiempo en abrir otra ventana de navegador, y buscar documentación, línea por línea, que como casi siempre es en inglés y en este caso con más proporción de palabras técnicas que no siempre son comunes.<br/>De esta forma se provee una forma más cómoda para su estudio.<br/>
<br/>
Competencia original: https://www.kaggle.com/c/histopathologic-cancer-detection<br/>
Notebook original: https://www.kaggle.com/code/leeking/cnn-conv2d-separableconv2d-keras-new-model-1

In [2]:
# Este entorno de Python 3 viene con varias librerías analíticas útiles instaladas
# Está definido por la imagen docker de kaggle/python: https://github.com/kaggle/docker-python
# Por ejemplo, tiene varios paquetes útiles para cargar.
# Los archivos de entrada están disponibles en la carpeta "../input/" o "/kaggle/input"

# Biblioteca de algebra lineal: vectores, matrices, funciones matemáticas de alto nivel...
import numpy as np 

 # Librería de procesamiento de datos, se usa por ejemplo para leer los archivos CSV I/O (ej: pd.read_csv)
import pandas as pd

# Permite ejecutar ciertos comandos aquí en el notebook, como si fueran ejecutados en una consola (shell)
import os 
 
# El módulo GLOB se usa para buscar rutas de archivos, de acuerdo a un patrón que sigue las reglas de la 
# consola (shell) de Unix
from glob import glob

# SHUFFLE (en inglés "barajar"), recibe una secuencia de items (como una lista) y reorganiza el orden.
from random import shuffle 
 
# OpenCV -> "OPEN Computer Vision", librería que se ocupa de "problemas" de reconocimiento de imágenes
import cv2

# PYPLOT viene de PYthon PLOT (plot = gráfico). pyplot es una interfaz basada en estados para MATPLOTLIB.
# Y MATPLOTLIB es una librería para generar gráficos en 2D. Son los que se usan al final de este notebook.
import matplotlib.pyplot as plt
    

# SKLEARN (Scikit-learn) es una librería de aprendizaje automático que provee versiones eficaces de algoritmos 
# comunes.
# MODEL_SELECTION es un método para establecer un "blueprint" (plano, diseño) para analizar datos y luego
# usarlo para medir nuevos datos. Construir un modelo adecuado mejora las predicciones.
# TRAIN_TEST_SPLIT (como su nombre lo declara)  sirve para dividir un dataset en dos: training y test. Este
# último se usa para evaluar la performance del modelo.
from sklearn.model_selection import train_test_split

# KERAS: biblioteca de redes neuronales
# PREPROCESSING: sirve para, partiendo de datos crudos almacenados en disco (hoy disco ya no es literal), a un
# dataset útil para entrenar un modelo.
# IMAGE: el preprocesamiento, específicamente para imágenes
# IMAGEDATAGENERATOR: genera lotes de datos de imágenes con aumento de datos (data augmentation) en tiempo real, 
# o sea en el momento.
from keras.preprocessing.image import ImageDataGenerator

# LAYERS (capas) son los bloques de construcción de redes neuronales en Keras. Se usarán para construir las 
# capas del modelo, a través de las cuáles el software interpretará las imágenes (con los parámetros -weights-
# que irá descubriendo).
# INPUT se usa para definir la entrada de datos al modelo: la estructura (shape) de los datos de cada imagen 
# (96,96,3).
from keras import layers, Input

# LOSSES (pérdidas) son un conjunto de funciones cuyo propósito es definir cómo se computa la "cantidad"
# (o monto) que el modelo debe buscar minimizar durante el entrenamiento.
# MAE (mean absolute error) calcula el promedio de la diferencia absoluta (o sea en módulo) entre las 
# predicciones y los labels reales.
# SPARSE_CATEGORICAL_CROSSENTROPY calcula la entropía cruzada que es un valor probabilístico calculado sumatoria
# y logaritmos.
# BINARY_CROSSENTROPY (entropía cruzada binaria) un cálculo similar al anterior, pero solo para labels binarios,
# como este.
from keras.losses import mae, sparse_categorical_crossentropy, binary_crossentropy

# keras.MODELS ofrece varias formas de crear modelos, que son el compilado de capas (layers) definidos aquí en 
# el notebook...
# MODEL es uno de estos modelos posibles a crear, que agrupa un conjunto (no necesariamente lineal / secuencial) 
# de capas creando un solo objeto con ellas.
from keras.models import Model

# keras.APPLICATIONS son modelos de aprendizaje profundo que se proveen (eventualmente) con parámetros (weights)
# pre entrenados.
# NASNET (Neural Architecture Search NETwork) es una arquitectura, una familia de modelos que fueron creados
# para datasets específicos, ya incluyen las capas (layers) de un modelo, de acuerdo a ciertos parámetros. Posee
# dos tipos de capas: normal y de reducción.
# PREPROCESS_INPUT realiza un preprocesamiento de un tensor o matriz Numpy que tienen codificado un lote de 
# imágenes, así adecua la imagen al formato que el modelo necesita.
# https://www.tensorflow.org/api_docs/python/tf/keras/applications/nasnet/preprocess_input
from keras.applications.nasnet import preprocess_input

# Corregido respecto al notebook original que los importaba de keras.optimizers
# Un OPTIMIZER (optimizador) se usa para mejorar la velocidad y performance para entrenar un modelo específico.
# ADAM (Adaptive Moment Estimation) agrega dos variables adicionales para cada variable a entrenar. El propósito
# es calcular las tasas de aprendizaje adaptativo para cada parámetro: diferentes partes de la red neuronal
# tienen diferente sensibilidad al ajuste de peso (weights).
# RMSprop: cada parámetro puede tener diferente tasa de aprendizaje, es como AdaGrad pero se agrega un 
# coeficiente de atenuación. RMSprop se basa en Rprop.
from tensorflow.keras.optimizers import Adam, RMSprop 

# CALLBACKS, tal como su nombre lo indica son llamadas a determinadas acciones en algún momento del entrenamiento,
# por ejemplo al inicio o fin de una "epoch", o luego de un lote (batch). Se pueden usar por ejemplo para loguear,
# guardar el modelo o detener antes de tiempo.
# MODELCHECKPOINT es un callback para grabar el modelo o sus pesos cada cierta frecuencia. Se define qué grabar,
# la frecuencia, etc.
# EARLYSTOPPING como su nombre lo sugiere, produce una interrupción temprana del entrenamiento cuando por ejemplo
# la pérdida (loss) ya no está disminuyendo (considerando, si aplica, min_delta y patience -número de epochs de 
# tolerancia-).
# REDUCE LR ON PLATEAU reduce la tasa de aprendizaje cuando un modelo, durante el entrenamiento, deja de mejorar.
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# IMGAUG es una librería para "data augmentation" en imágenes. El aumento de imágenes consiste en generar más 
# variantes de las mismas imágenes, aumentando así el dataset de entrenamiento. Se hacen variaciones, rotaciones,
# se quitan de foco, cambian algunos colores, etc., manteniendo el mismo label.
# Más info: https://albumentations.ai/docs/introduction/image_augmentation/
from imgaug import augmenters as iaa

# Librería de aumento de imágenes
import imgaug as ia

# Lee la carpeta "../input" relativa a la carpeta actual e imprime su contenido en pantalla.
print(os.listdir("../input"))

# leo planilla de entrenamiento
df_train = pd.read_csv("../input/histopathologic-cancer-detection/train_labels.csv")

# Cualquier resultado grabado en el directorio actual se graba en '/output'

['histopathologic-cancer-detection']


In [3]:
# La función ZIP devuelve justamente un objeto ZIP que nada tiene que ver con compresión de datos, sino que es un 
# iterador de tuplas, donde los elementos (en este caso) df_train.id y df_train.label se van devolviendo a pares.
# Lo de K:V FOR K,V genera un "diccionario" con todos esos pares, como una lista tipo clave-valor.
id_label_map = {k:v for k,v in zip(df_train.id.values, df_train.label.values)}

# El conocido HEAD() devuelve los 5 primeros items del dataframe, no se para qué explico esto! :)
df_train.head()


,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [4]:
# Función que como su nombre dice, devuelve el id (de imagen), partiendo de la ruta y nombre de uno de los 
# archivos. OS.PATH.SEP es el "SEParador para este sistema operativo (OS) de rutas (PATH). Toma el final (-1) de
# los elementos separados (SPLIT) de la ruta y le quita el ".tif" con la función REPLACE. Como los ids archivos
# son una cadena alfanumérica de 40 caracteres con extensión tif, devuelve solo esos 40 caracteres, que son el 
# id.
def get_id_from_file_path(file_path):
    return file_path.split(os.path.sep)[-1].replace('.tif', '')

In [5]:
# Esto demora un poquito... carga los nombres de los archivos existentes "en disco", en las variables.
labeled_files = glob('../input/histopathologic-cancer-detection/train/*.tif')
test_files = glob('../input/histopathologic-cancer-detection/test/*.tif')

# Se muestra en pantalla los totales de archivos leídos.
print("labeled_files size :", len(labeled_files))
print("test_files size :", len(test_files))

labeled_files size : 220025
test_files size : 57458


In [6]:
# Se divide el set de entrenamiento en dos subconjuntos: entrenamiento y validacion. El RANDOM_STATE es una forma
# de efectuar una selección al azar, pero que será siempre la misma cada vez que se ejecute. ¿Se entiende?
train, val = train_test_split(labeled_files, test_size=0.1, random_state=101010)

In [2]:
# CHUNKER lo que hace es recibir un dataframe y recortarlo en porciones de tamaño SIZE, devolviendo tuplas en 
# las que cada una contiene una de esas esas "porciones".
# RANGE(0, len(seq), size) crea una secuencia de números desde 0, hasta la cantidad de elementos en SEQ (menos 
# uno), pero los incrementos son de SIZE. Ejemplo: 0, 3, 6, 9 ...
# SEC[pos:pos + size] corta una secuencia arrancando desde uno de los números devueltos por RANGE, de largo SIZE.
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


# Esta función genera (y retorna) una secuencia de aumentadores (data augmentation) para aplicarle a imágenes.
def get_seq():
    
    # Se define la variable SOMETIMES que luego se usa en forma de función para que no todos los 
    # aumentadores se apliquen, de esa forma se logra una variedad en las transformaciones y se suaviza el
    # efecto (comparado con que se apliquen todas juntas a una misma imagen) así la imagen no se deforma
    # tanto que ya no sirva.
    # https://imgaug.readthedocs.io/en/latest/source/examples_basics.html#heavy-augmentations
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    
    # Se define una secuencia de aumentadores
    seq = iaa.Sequential(
        [
            # https://imgaug.readthedocs.io/en/latest/source/api_augmenters_flip.html
            iaa.Fliplr(0.5), # Invertir horizontalmente el 50% de todas las imágenes
            iaa.Flipud(0.2), # Invertir verticalmente el 50% de todas las imágenes
            
            # Guía en: https://imgaug.readthedocs.io/en/latest/source/overview/geometric.html
            sometimes(iaa.Affine(
                # Cambiar tamaño de imágenes a 80-120% de su tamaño, individualmente, por eje
                scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}, 
                
                # Mover entre -20 y +20 % las imágenes independiéntemente, por eje
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, 
                
                rotate=(-10, 10), # Rotar entre -45 y +45 grados
                
                # Torcer (es como rotar pero dejando la base y tope horizontales) entre -16 y +16 grados
                shear=(-5, 5), 
                
                order=[0, 1], # Usar "nearest neighbour" o "bilinear interpolation" para el resampleado (rápido)
                cval=(0, 255), # Si el modo es constante, usar un cval entre 0 y 255 (equivalente a ALL)
                mode=ia.ALL # Usar alguno de los modos de deformación de imágenes de scikit
            )),
            
            # Ejecutar entre 0 y 5 de los siguientes (menos importantes) aumentadores (augmenters) por imagen.
            # No ejecutar todos, porque eso habitualmente sería muy fuerte.
            iaa.SomeOf((0, 5),
                [
                    # Convertir imágenes a su representación en superpíxeles
                    # ¿Qué son los superpíxeles? 
                    # https://www.tu-chemnitz.de/etit/proaut/en/research/superpixel.html
                    sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))),                     
                    
                    # ONEOF ejecuta azarósamente solo uno (cada vez) de los métodos listados dentro.
                    # https://imgaug.readthedocs.io/en/latest/source/overview/meta.html#oneof
                    iaa.OneOf([
                        # Guía para los 3 en https://imgaug.readthedocs.io/en/latest/source/api_augmenters_blur.html
                        
                        # Borronear imágenes con un sigma entre 0 y 3.0
                        iaa.GaussianBlur((0, 1.0)),
                        
                        # Borronear imagen usando promedios locales con tamaños de kernel entre 2 y 7
                        iaa.AverageBlur(k=(3, 5)), 
                        
                        # Borronear imagen usando medianas locales con tamaños de kernel entre 2 y 7
                        iaa.MedianBlur(k=(3, 5)), 
                    ]),
                    
                    # Mejorar nitidez de imagen
                    # https://imgaug.readthedocs.io/en/latest/source/overview/convolutional.html#sharpen
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)), 
                    
                    # Mejorar nitidez de imagen
                    # https://imgaug.readthedocs.io/en/latest/source/overview/convolutional.html#emboss
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # Estampar imagen
                    
                    # Buscar todos los bordes o bien los bordes dirigidos,
                    # mezclar el resultado con la imagen original usando una máscara de manchas.
                    # Explicación del mezclado: https://imgaug.readthedocs.io/en/latest/source/alpha.html
                    iaa.SimplexNoiseAlpha(iaa.OneOf([
                        # Resalta los bordes
                        # https://imgaug.readthedocs.io/en/latest/source/overview/convolutional.html#edgedetect                        
                        iaa.EdgeDetect(alpha=(0.5, 1.0)),
                        
                        # Resalta bordes con una dirección determinada
                        # https://imgaug.readthedocs.io/en/latest/source/overview/convolutional.html#directededgedetect
                        iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                    ])),
                    
                    # Agregar "ruido" de acuerdo a una distribución gaussiana
                    # https://imgaug.readthedocs.io/en/latest/source/overview/arithmetic.html#additivegaussiannoise
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.5),
                    
                    iaa.OneOf([
                        # Azarosamente remueve hasta el 10% de los pixeles (pasan a ser píxeles negros)
                        # https://imgaug.readthedocs.io/en/latest/source/overview/arithmetic.html#additivegaussiannoise
                        iaa.Dropout((0.01, 0.05), per_channel=0.5), 
                        
                        # Similar al dropout de arriba pero en lugar de píxeles aislados, lo hace en 
                        # varios grupos que forman pequeños rectángulos.
                        # https://imgaug.readthedocs.io/en/latest/source/overview/arithmetic.html#coarsedropout
                        iaa.CoarseDropout((0.01, 0.03), size_percent=(0.01, 0.02), per_channel=0.2),
                    ]),
                    
                    # Invierte canales de color de acuerdo a parámetros. 
                    # https://imgaug.readthedocs.io/en/latest/source/overview/arithmetic.html#invert
                    iaa.Invert(0.01, per_channel=True),
                    
                    # Cambia el brillo (agrega valores a los píxeles), entre -2 y 2
                    # https://imgaug.readthedocs.io/en/latest/source/api_augmenters_arithmetic.html#imgaug.augmenters.arithmetic.Add
                    iaa.Add((-2, 2), per_channel=0.5), 
                    
                    # Decrementa o incrementa tonalidad y saturación.
                    # https://imgaug.readthedocs.io/en/latest/source/overview/color.html#addtohueandsaturation
                    iaa.AddToHueAndSaturation((-1, 1)), 
                    
                    iaa.OneOf([
                        # Multiplica todos los píxels de la imagen, por un valor aleatorio dentro del rango
                        # parametrizado, el mismo valor para toda la imagen
                        # https://imgaug.readthedocs.io/en/latest/source/api_augmenters_arithmetic.html#imgaug.augmenters.arithmetic.Multiply
                        iaa.Multiply((0.9, 1.1), per_channel=0.5),
                        
                        # Alpha se refiere a la transparencia de una porción de una imagen (desde un píxel a toda
                        # la imagen). Este aumentador toma una imagen, le hace un efecto como el de bordes en 
                        # sobrerelieve (en toda una imagen en blanco/negro) y luego la fusiona con la imagen 
                        # original (blend) en modo alpha (trasparencias, no aditivo) usando una máscara de ruido
                        # de frecuencia.
                        # https://imgaug.readthedocs.io/en/latest/source/alpha.html#frequencynoisealpha
                        iaa.FrequencyNoiseAlpha(
                            exponent=(-1, 0),
                            first=iaa.Multiply((0.9, 1.1), per_channel=True),
                            second=iaa.ContrastNormalization((0.9, 1.1))
                        )
                    ]),
                    
                    # Como su nombre sugiere "elastic", mueve pixels localmente alrededor de su posición original, 
                    # con fuerzas aleatorias.
                    # https://imgaug.readthedocs.io/en/latest/source/overview/geometric.html#elastictransformation
                    sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), 
                    
                    # Piecewise: por partes, afine: que conserva mayormente la dirección de las lineas pero
                    # puede alternar su distancia y ángulos.
                    # Genera distorciones locales que se ven como áreas "torcidas" en la imagen.
                    # https://imgaug.readthedocs.io/en/latest/source/overview/geometric.html#piecewiseaffine
                    sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                    
                    # "Transforma la perspectiva": coloca 4 puntos, uno cerca de cada esquina de la imagen, a
                    # distancias aleatorias dentro de un rango, formando así un polígono, luego recorta la imagen
                    # por los bordes del polígono y mueve esos nuevos bordes para volver a formar un cuadrado. 
                    # https://imgaug.readthedocs.io/en/latest/source/overview/geometric.html#perspectivetransform
                    sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                ],
                random_order=True # Los aumentadores se ejecutan en orden aleatorio
            )
        ],
        random_order=True # Los aumentadores (en grupos o individuales) se ejecutan en orden aleatorio
    )
    return seq


# Función que devuelve las imágenes originales ya leídas de disco más todas sus variantes aumentadas (si asi se pidió
# por parámetro) ya preprocesadas, junto con el label de cada una, para iterar todas una vez.
# list_files: los archivos encontrados en "disco"
# id_label_map: el iterador de tuplas, del dataset de entrenamiento (id -> label)
# batch_size: tamaño del lote
# augment: si se aplican o no los aumentadores de imágenes
def data_gen(list_files, id_label_map, batch_size, augment = False):
    seq = get_seq() # La secuencia de aumentadores
    while True:
        shuffle(list_files) # Mezclar el listado
        for batch in chunker(list_files, batch_size): # Por cada lote, cortado del tamaño batch_size
            X = [cv2.imread(x) for x in batch] # Se lee cada imagen desde "disco"
            Y = [id_label_map[get_id_from_file_path(x)] for x in batch] # Se lee el label
            if augment:
                X = seq.augment_images(X) # Se generan las variantes adicionales de las imágenes
            X = [preprocess_input(x) for x in X] # Se preprocesan las imágenes originales y sus variantes
                
            yield np.array(X), np.array(Y) # Devuelve un generador de la lista de imagenes, para iterar una vez.
    

In [8]:
def get_model_classif_nasnet():  
    # epoch = 20 ---- 95.87%
    inputs = Input((96, 96, 3)) # Ya explicado en el primer bloque de código

    # Las redes convolucionales, basadas en la biología de la visión, toman grupos de píxeles cercanos y van
    # operando, calculando datos y pasándoselos de capa en capa para obtener así patrones / conclusiones sobre
    # la imagen.
    # Arranca con un filtro convolucional en 2 dimensiones, es una capa del modelo, según los parámetros esta 
    # tendrá 32 filtros y un "stride" (paso) de 3 píxeles, o sea que se va moviendo de a 3 píxeles cada vez que
    # va recorriendo la imagen de entrada. Producirá 32 matrices de salida. El padding -> 'same' significa que
    # el tamaño de las matrices de salida no se reducirán por aplicar padding alguno.
    # https://keras.io/api/layers/convolution_layers/convolution2d/
    x1 = layers.Conv2D(32, 3, padding='same')(inputs) 
    
    # Se normaliza la capa anterior, normalización de lote es transformar los valores obtenidos para que 
    # promedien 0, y que la desviación estandar sea 1. La variable 'x1' es la entrada de esta capa, que a su 
    # vez se sobreescribe con la salida.
    # https://keras.io/api/layers/normalization_layers/batch_normalization/
    x1 = layers.BatchNormalization()(x1)
    
    # Se aplica la función de activación RELU a la salida de la capa previa.
    # Una función de activación recibe varias entradas y devuelve una salida transformando la combinación de 
    # entradas, pesos y sesgos para la siguiente capa del modelo. 
    # Se pueden usar varias funciones, en este caso se usa RELU -> REctified Linear activation Unit. En 
    # español: Función Lineal de Activación Rectificada.
    # https://keras.io/api/layers/activations/
    x1 = layers.Activation('relu')(x1)
    
    # Otra capa convolucional
    x1 = layers.Conv2D(32,3,padding='same')(x1)
    
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.Conv2D(32,3,padding='same')(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    
    # Max-Pooling, para secciones de la imagen se obtiene la feature más importante. Es otra manera de obtener
    # información local, aparte de la capa convolucional. En este caso el tamaño de salida es 2x2 y la entrada
    # es la capa previa, de mayor tamaño.
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
    x1 = layers.MaxPool2D(2,2)(x1)  

    # Se siguen aplicando capas ya explicadas
    x2 = layers.Conv2D(64,3,padding='same')(x1)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.Activation('relu')(x2)
    x2 = layers.Conv2D(64,3,padding='same')(x2)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.Activation('relu')(x2)
    x2 = layers.Conv2D(64,3,padding='same')(x2)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.Activation('relu')(x2)
    residual_x1 = layers.Conv2D(64,1,strides=1,padding='same')(x1)
    
    # Suma dos capas y devuelve una.
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add
    x2 = layers.add([x2,residual_x1])
    
    x2 = layers.MaxPool2D(2,2)(x2)
    
    # Una convolución que se realiza de manera independiente por cada uno de los canales (en este caso 3) de
    # la entrada, y cada matriz resultante se forma juntando los 3 canales resultantes.
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/SeparableConv2D
    # https://paperswithcode.com/method/depthwise-convolution
    x2_s = layers.SeparableConv2D(64,3,padding='same')(x2)
    
    x2_s = layers.BatchNormalization()(x2_s)
    x2_s = layers.Activation('relu')(x2_s)
    x2_s = layers.SeparableConv2D(64,3,padding='same')(x2_s)
    x2_s= layers.BatchNormalization()(x2_s)
    x2_s= layers.Activation('relu')(x2_s)
    x2_s = layers.SeparableConv2D(64,3,padding='same')(x2_s)
    x2_s= layers.BatchNormalization()(x2_s)
    x2_s= layers.Activation('relu')(x2_s)
    x2_s = layers.add([x2_s,x2]) 
    x2_s = layers.MaxPool2D(2,2)(x2_s)
    
    x3 = layers.Conv2D(128,3,padding='same')(x2)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.Activation('relu')(x3)
    x3 = layers.Conv2D(128,3,padding='same')(x3)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.Activation('relu')(x3)
    x3 = layers.Conv2D(128,3,padding='same')(x3)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.Activation('relu')(x3)
    residual_x2 = layers.Conv2D(128,1,strides=1,padding='same')(x2)
    x3 = layers.add([residual_x2,x3]) 
    
    x3_x3 = layers.Conv2D(128,3,padding='same')(x3)
    x3_x3 = layers.BatchNormalization()(x3_x3)
    x3_x3 = layers.Activation('relu')(x3_x3)
    x3_x3 = layers.Conv2D(128,3,padding='same')(x3_x3)
    x3_x3 = layers.BatchNormalization()(x3_x3)
    x3_x3 = layers.Activation('relu')(x3_x3)
    x3_x3 = layers.Conv2D(128,3,padding='same')(x3_x3)
    x3_x3 = layers.BatchNormalization()(x3_x3)
    x3_x3 = layers.Activation('relu')(x3_x3)
    x3_x3 = layers.add([x3,x3_x3]) 
    x3_x3 = layers.MaxPool2D(2,2)(x3_x3)
    
    # Concatena las matrices de entrada.
    # https://keras.io/api/layers/merging_layers/concatenate/
    concetenated_1 = layers.concatenate([x3_x3,x2_s])
    
    x3_s = layers.SeparableConv2D(128,3,padding='same')(concetenated_1)
    x3_s = layers.BatchNormalization()(x3_s)
    x3_s = layers.Activation('relu')(x3_s)
    x3_s = layers.SeparableConv2D(128,3,padding='same')(x3_s)
    x3_s= layers.BatchNormalization()(x3_s)
    x3_s= layers.Activation('relu')(x3_s)
    x3_s = layers.SeparableConv2D(128,3,padding='same')(x3_s)
    x3_s= layers.BatchNormalization()(x3_s)
    x3_s= layers.Activation('relu')(x3_s)
    x3_s = layers.add([x3_s,x3_x3]) 
    x3_s = layers.MaxPool2D(2,2)(x3_s)
    
    x4 = layers.Conv2D(256,3,padding='same')(x3_x3)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.Activation('relu')(x4)
    x4 = layers.Conv2D(256,3,padding='same')(x4)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.Activation('relu')(x4)
    x4 = layers.Conv2D(256,3,padding='same')(x4)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.Activation('relu')(x4)
    residual_x3 = layers.Conv2D(256,1,strides=1,padding='same')(x3_x3)
    x4 = layers.add([residual_x3,x4]) 
    
    x4_x4 = layers.Conv2D(256,3,padding='same')(x4)
    x4_x4 = layers.BatchNormalization()(x4_x4)
    x4_x4 = layers.Activation('relu')(x4_x4)
    x4_x4 = layers.Conv2D(256,3,padding='same')(x4_x4)
    x4_x4 = layers.BatchNormalization()(x4_x4)
    x4_x4 = layers.Activation('relu')(x4_x4)
    x4_x4 = layers.Conv2D(256,3,padding='same')(x4_x4)
    x4_x4 = layers.BatchNormalization()(x4_x4)
    x4_x4 = layers.Activation('relu')(x4_x4)
    x4_x4 = layers.add([x4,x4_x4]) 
    x4_x4 = layers.MaxPool2D(2,2)(x4_x4)

    concetenated_2 = layers.concatenate([x4_x4,x3_s])
    x4_s = layers.SeparableConv2D(256,3,padding='same')(concetenated_2)
    x4_s = layers.BatchNormalization()(x4_s)
    x4_s = layers.Activation('relu')(x4_s)
    x4_s = layers.SeparableConv2D(256,3,padding='same')(x4_s)
    x4_s= layers.BatchNormalization()(x4_s)
    x4_s= layers.Activation('relu')(x4_s)
    x4_s = layers.SeparableConv2D(256,3,padding='same')(x4_s)
    x4_s= layers.BatchNormalization()(x4_s)
    x4_s= layers.Activation('relu')(x4_s)
    x4_s = layers.add([x4_s,x4_x4]) 
    x4_s = layers.MaxPool2D(2,2)(x4_s)
    
    x5 = layers.Conv2D(512,3,padding='same')(x4_x4)
    x5 = layers.BatchNormalization()(x5)
    x5 = layers.Activation('relu')(x5)
    x5 = layers.Conv2D(512,3,padding='same')(x5)
    x5 = layers.BatchNormalization()(x5)
    x5 = layers.Activation('relu')(x5)
    x5 = layers.Conv2D(512,3,padding='same')(x5)
    x5 = layers.BatchNormalization()(x5)
    x5 = layers.Activation('relu')(x5)
    residual_x4 = layers.Conv2D(512,1,strides=1,padding='same')(x4_x4)
    x5 = layers.add([residual_x4,x5])

    x5_x5 = layers.Conv2D(512,3,padding='same')(x5)
    x5_x5 = layers.BatchNormalization()(x5_x5)
    x5_x5 = layers.Activation('relu')(x5_x5)
    x5_x5 = layers.Conv2D(512,3,padding='same')(x5_x5)
    x5_x5 = layers.BatchNormalization()(x5_x5)
    x5_x5 = layers.Activation('relu')(x5_x5)
    x5_x5 = layers.Conv2D(512,3,padding='same')(x5_x5)
    x5_x5 = layers.BatchNormalization()(x5_x5)
    x5_x5 = layers.Activation('relu')(x5_x5)
    x5_x5 = layers.add([x5,x5_x5])
    x5_x5 = layers.MaxPool2D(2,2)(x5_x5)
    
    concetenated_3 = layers.concatenate([x5_x5,x4_s])
    x5_s = layers.SeparableConv2D(512,3,padding='same')(concetenated_3)
    x5_s = layers.BatchNormalization()(x5_s)
    x5_s = layers.Activation('relu')(x5_s)
    x5_s = layers.SeparableConv2D(512,3,padding='same')(x5_s)
    x5_s= layers.BatchNormalization()(x5_s)
    x5_s= layers.Activation('relu')(x5_s)
    x5_s = layers.SeparableConv2D(512,3,padding='same')(x5_s)
    x5_s= layers.BatchNormalization()(x5_s)
    x5_s= layers.Activation('relu')(x5_s)
    x5_s = layers.add([x5_s,x5_x5]) 

    # Global Average (promedio) Pooling calcula la salida promedio de cada mapa de características de la capa
    # previa. Esta simple operación reduce significativamente los datos y prepara el modelo para la capa de
    # clasificación final. No tiene parámetros entrenables.
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling2D
    # https://adventuresinmachinelearning.com/global-average-pooling-convolutional-neural-networks/
    x = layers.GlobalAveragePooling2D()(x5_s)

    # Capa usada en las etapas finales de la red neuronal, que cambia la dimensionalidad de la salida para que
    # el modelo pueda definir la relación entre las características detectadas y los labels. 64 es la dimensión
    # del vector de salida.
    # https://keras.io/api/layers/core_layers/dense/
    # https://analyticsindiamag.com/a-complete-understanding-of-dense-layers-in-neural-networks/
    x = layers.Dense(64)(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # El 20% de las neuronas (ya que el parámetro es 0.2) elegidas aleatoramente se ignoran, y el otro 80% se
    # divide por 0.8 (o sea se incrementa) para que el promedio no se altere. El propósito de ignorar una
    # porción es evitar que el modelo haga overfitting (sobreajuste).
    # https://keras.io/api/layers/regularization_layers/dropout/
    # https://www.aprendemachinelearning.com/que-es-overfitting-y-underfitting-y-como-solucionarlo/
    x = layers.Dropout(0.2)(x)
    
    # La función sigmoide es otra posible función de activación.
    # https://es.wikipedia.org/wiki/Funci%C3%B3n_sigmoide
    # El objetivo es por ejemplo definir si un 0,7 es un SI o un NO, lo mismo con cualquier valor como 0,1...
    # 0,8... etc. Para obtener así la predicción booleana de si la imágen es tumoral o no.
    output_tensor = layers.Dense(1, activation='sigmoid')(x)

    # Aquí se le crea el objeto de modelo con las capas definidas
    model = Model(inputs,output_tensor)
    
    # Configura el modelo para el entrenamiento. Se define el optimizador, la función de pérdida y las métricas
    # (acc) a evaluar durante entrenamiento y testing
    # https://keras.io/api/models/model_training_apis/
    model.compile(optimizer=Adam(0.01), loss=binary_crossentropy, metrics=['acc'])
    
    # Muestra en pantalla la estructura terminada del modelo, con sus capas, parámetros, etc.
    model.summary()
    
    return model


In [9]:
model = get_model_classif_nasnet()

2022-12-12 00:19:20.881805: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 96, 96, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 96, 96, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 96, 96, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
## Primera ronda

batch_size = 128 # Tamaño de los lotes (grupos de imágenes)
h5_path = "./model_2.h5"

# ModelCheckpoint (función callback) se usa con las funciones de entrenamiento de modelo (model.fit o
# model.fit_generator) para grabar los pesos (weights, que junto con los biases son los parámetros que aprende el
# modelo) generados por el entrenamiento, en un archivo, cada cierto intervalo, de modo que el modelo o los pesos
# (weights) pueden ser leídos en otro momento para continuar el entrenamiento desde ese punto de avance.
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# Se realiza el entrenamiento en sí. Se usa fit_generator (y no fit) cuando el dataset es muy grande y no entra
# en memoria. El método retorna algunos valores del entrenamiento.
# https://www.geeksforgeeks.org/keras-fit-and-keras-fit_generator/
history = model.fit_generator(
    data_gen(train, id_label_map, batch_size, augment = True), # Materia prima para entrenar: imágenes.
    validation_data = data_gen(val, id_label_map, batch_size), # Las imágenes para validar
    epochs = 15, # Cuántas pasadas hará el modelo por cada set de datos
    verbose = 1, # Qué se mostrará mientras se entrena: 1 = barra de progreso
    callbacks = [checkpoint], # Ya explicado arriba
    steps_per_epoch = len(train), # Tamaño de los lotes de entrenamiento,
    validation_steps = len(val)) # Tamaño de los lotes de validación)

# Los pesos (weights) y los biases (sesgos) son los parámetros que "aprende" el modelo luego de entrenar, con los
# que luego puede hacer predicciones. Aquí se graban los pesos, para todas las capas del modelo. De esta forma se
# puede suspender la ejecución al final de este bloque código y en otro momento cargarlos y retomar la ejección
# del siguiente bloque.
# https://keras.io/api/models/model_saving_apis/#saveweights-method
model.save_weights(h5_path)

In [ ]:
# Se verifica que se haya grabado el archivo, listando el contenido de la carpeta.
os.listdir('../input/')

In [ ]:
## Segunda ronda

h5_path = "./model_2.h5"
model.load_weights(h5_path)
batch_size = 128
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

history = model.fit_generator(
    data_gen(train, id_label_map, batch_size, augment=True),
    validation_data=data_gen(val, id_label_map, batch_size),
    epochs=15, verbose=1,
    callbacks = [checkpoint],
    steps_per_epoch = len(train),
    validation_steps = len(val))

model.save_weights(h5_path)

In [ ]:
# Tercera ronda

h5_path = "./model_2.h5"
model.load_weights(h5_path)
batch_size = 128
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

history = model. (
    data_gen(train, id_label_map, batch_size, augment=True),
    validation_data = data_gen(val, id_label_map, batch_size),
    epochs = 5, verbose = 1,
    callbacks = [checkpoint],
    steps_per_epoch = len(train),
    validation_steps = len(val))

model.save_weights(h5_path)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# Se muestran 2 gráficos de resultados de entrenamiento y validación.
# El primer parámetro son las coordenadas en X, el segundo las coordenadas en Y, 
# el tercero el tipo de formato, como color, si son puntos o líneas... (b = blue, o = circulitos), se trazan dos
# secuencias por gráfico, uno con líneas otro con puntos (círculos).
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html
plt.plot(epochs, acc, 'bo', label='Train acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')

# Prepara la leyenda, que es el cartel dentro del gráfico que dice a qué corresponde cada secuencia (una línea,
# o secuencia de puntos)
# https://matplotlib.org/stable/api/legend_api.html
plt.legend()

# Comienza otro gráfico.
# https://matplotlib.org/stable/api/figure_api.html
plt.figure()

plt.plot(epochs, loss, 'bo', label='Train loss')
plt.plot(epochs, val_loss,'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

# Muestra los gráficos que se prepararon.
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.show.html
plt.show()

In [ ]:
preds = [] # Vector donde se guardarán todas las predicciones
ids = []

In [ ]:
# A predecir, amigos!

model.load_weights(h5_path) # Se cargan los parámetros ya entrenados

# Se divide el dataset de test en lotes, y por cada uno...
for batch in chunker(test_files, batch_size):
    # Preprocesamiento de las imágenes, leyéndolas primero de disco, ya que 'test_files' contiene solo nombres
    # de archivo.
    X = [preprocess_input(cv2.imread(x)) for x in batch]
    
    # Obtiene los IDs alfanuméricos, a partir de los nombres de archivos
    ids_batch = [get_id_from_file_path(x) for x in batch]
    
    # Se convierte el lote a un array de la librería NumPy (NUMerical PYthon)
    X = np.array(X)
    
    # Aquí se almacenarán las predicciones para este lote
    preds_batch = (
                    (
                        # model.predict, imposible no intuirlo por el nombre, realiza las predicciones de lo
                        # que se le envía en X.
                        # ravel() cambia la forma de la matriz por un vector en 1 dimensión.
                        # https://keras.io/api/models/model_training_apis/
                        # ::-1 se relaciona con start:stop:step, y en este caso implica cambiar el sentido de
                        # recorrido por el inverso.
                        # ¿Por qué se predice 4 veces, mandándole esos [:, ::-1, :, :] raros?
                        # Primero predice las imágenes tal como están, luego las predice invirtiendo el orden
                        # de las filas (como poniéndoles un espejo encima), luego las predice invirtiendo el
                        # orden de las filas y de las columnas, y por último solo invirtiendo el orden de las
                        # columnas (el orden de los colores, 4ta dimensión, nunca se invierte).
                        model.predict(X).ravel() * model.predict(X[:, ::-1, :, :]).ravel()
                        * model.predict(X[:, ::-1, ::-1, :]).ravel()
                        * model.predict(X[:, :, ::-1, :]).ravel() 
                    ) 
        
                    # El doble asterisco es para calcular potencias. Lo eleva a 1/4 para compensar que
                    # previamente se multiplicaron 4 predicciones entre si.
                    ** 0.25         
    
                # Convierte el array a una lista ordinaria
                # https://www.educative.io/answers/what-is-the-array-tolist-function-in-python
                ).tolist() 
    
    
    preds += preds_batch # Incorpora las predicciones de este lote al listado final.
    ids += ids_batch # Incorpora los ids de este lote al listado final

In [ ]:
# Se genera un dataframe con los ids y las predicciones
df = pd.DataFrame({'id':ids, 'label':preds})

# Se guarda en archivo CSV, que es el que se presentó a la competencia
df.to_csv("baseline_nasnet.csv", index=False)

# Se muestran las 5 primeras líneas del resultado
df.head()